In [142]:
import librosa
import numpy as np
import pandas as pd
import soundfile as sf

## Dataset creation

In [143]:
import os
import librosa
import soundfile as sf

# Assuming 'output_filename' is the desired filename for the output WAV file
output_filename = 'long_form_audio/concatenated_output.wav'
data=pd.read_csv('../MADHAV_task_1/Task3/metadata_test_clean.tsv', sep='\t', header=None, names=['path', 'text', 'num','sr'])
# List to store audio chunks
audio_chunks = []
target=""
# Iterate over all files in the directory
for i in range(10):
    file_path = f"output/audio_noise{i}.wav"

    # Load the audio file
    audio, sr = librosa.load(file_path, sr=None)
    if(target==""):
        target=data['text'][i]
    else:
        target+=" "+data['text'][i]
    # Append the audio to the list
    audio_chunks.append(audio)

# Concatenate all audio chunks
concatenated_audio = np.concatenate(audio_chunks)

# Save the concatenated audio to a new WAV file
sf.write(output_filename, concatenated_audio, samplerate=sr)
print(target)

she was tired of other things she tried this morning an air or two upon the piano sang a simple song in a sweet but slightly metallic voice and then seating herself by the open window read philips letter well mother said the young student looking up with a shade of impatience i hope thee told the elders that father and i are responsible for the piano and that much as thee loves music thee is never in the room when it is played i heard father tell cousin abner that he was whipped so often for whistling when he was a boy that he was determined to have what compensation he could get now thy ways greatly try me ruth and all thy relations is thy father willing thee should go away to a school of the worlds people i have not asked him ruth replied with a look that might imply that she was one of those determined little bodies who first made up her own mind and then compelled others to make up theirs in accordance with hers mother im going to study medicine margaret bolton almost lost for a mo

## Model algorithm

In [144]:
def split_audio_using_VAD(input_file, pred, chunk_length_sec=15):
    # Load the audio file
    audio, sr = librosa.load(input_file, sr=None)
    timestamps_df=pred*sr/1000
    # Calculate the total duration of the audio in samples
    total_samples = len(audio)

    # Calculate the chunk length in samples
    chunk_length_samples = int(sr * chunk_length_sec)

    # Create an array to store audio chunks
    audio_chunks = []
    split_sec=[]
    k=0
    index=0
    while k+chunk_length_samples<total_samples:
        start_sample = k
        end_sample = k+chunk_length_samples
        nearest_timestamp_index = np.abs(np.array(timestamps_df['end']>start_sample) - end_sample).argmin()
        nearest_timestamp_end_sample = int(timestamps_df.loc[nearest_timestamp_index, 'end'])
        end_sample = min(end_sample, nearest_timestamp_end_sample)
        chunk = audio[start_sample:end_sample]
        audio_chunks.append(chunk)
        split_sec.append((index, start_sample/sr, end_sample/sr))
        k=end_sample
        index+=1

    last_chunk = audio[k:total_samples]
    audio_chunks.append(last_chunk)
    split_sec.append((index, k/sr, total_samples/sr))
    split_sec=pd.DataFrame(split_sec, columns=['index', 'start', 'end'])
    return audio_chunks, split_sec

In [145]:
from fsmnvad import FSMNVad
# Specify the input audio file
input_file = "long_form_audio/concatenated_output.wav"
vad = FSMNVad()
segments = vad.segments_offline(input_file)
df=pd.DataFrame(segments, columns=['start', 'end'])
df

,start,end
0,1260,2300
1,2580,3200
2,3730,4680
3,4960,5650
4,6170,7660
5,8050,10490
6,10770,12230
7,12550,13170
8,13700,14770
9,15850,18100


In [146]:
# Split the audio into 15-second chunks with adjustment for the last chunk
chunks_array, split_sec = split_audio_using_VAD(input_file, df, 5)

# Now, chunks_array contains the audio chunks as numpy arrays
for i, chunk in enumerate(chunks_array):
    print(f"Chunk {i}: start: {split_sec['start'][i]} end: {split_sec['end'][i]}")

Chunk 0: start: 0.0 end: 2.3
Chunk 1: start: 2.3 end: 3.2
Chunk 2: start: 3.2 end: 4.68
Chunk 3: start: 4.68 end: 5.65
Chunk 4: start: 5.65 end: 7.66
Chunk 5: start: 7.66 end: 10.49
Chunk 6: start: 10.49 end: 12.23
Chunk 7: start: 12.23 end: 13.17
Chunk 8: start: 13.17 end: 14.77
Chunk 9: start: 14.77 end: 18.1
Chunk 10: start: 18.1 end: 23.1
Chunk 11: start: 23.1 end: 26.62
Chunk 12: start: 26.62 end: 31.36
Chunk 13: start: 31.36 end: 32.59
Chunk 14: start: 32.59 end: 36.84
Chunk 15: start: 36.84 end: 39.18
Chunk 16: start: 39.18 end: 41.37
Chunk 17: start: 41.37 end: 43.6
Chunk 18: start: 43.6 end: 44.85
Chunk 19: start: 44.85 end: 49.85
Chunk 20: start: 49.85 end: 52.16
Chunk 21: start: 52.16 end: 53.69
Chunk 22: start: 53.69 end: 57.32
Chunk 23: start: 57.32 end: 59.36
Chunk 24: start: 59.36 end: 61.14
Chunk 25: start: 61.14 end: 66.14
Chunk 26: start: 66.14 end: 66.69
Chunk 27: start: 66.69 end: 69.25
Chunk 28: start: 69.25 end: 71.76
Chunk 29: start: 71.76 end: 72.87
Chunk 30: st

In [147]:
!pip install -q espnet_model_zoo

In [148]:
#@title Choose English ASR model { run: "auto" }

lang = 'en'
fs = 16000 #@param {type:"integer"}
tag = 'Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave' #@param ["Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave", "kamo-naoyuki/librispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_bpe5000_scheduler_confwarmup_steps40000_optim_conflr0.0025_sp_valid.acc.ave"] {type:"string"}

In [149]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    **d.download_and_unpack(tag),
    device="cpu",
    minlenratio=0.0,
    maxlenratio=0.0,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

2023-12-11 18:13:15,181 WARNING [conformer_encoder.py:139] Using legacy_rel_pos and it will be deprecated in the future.
2023-12-11 18:13:15,181 WARNING [conformer_encoder.py:139] Using legacy_rel_pos and it will be deprecated in the future.
2023-12-11 18:13:15,181 WARNING [conformer_encoder.py:139] Using legacy_rel_pos and it will be deprecated in the future.
2023-12-11 18:13:15,181 WARNING [conformer_encoder.py:139] Using legacy_rel_pos and it will be deprecated in the future.
2023-12-11 18:13:15,181 WARNING [conformer_encoder.py:139] Using legacy_rel_pos and it will be deprecated in the future.
2023-12-11 18:13:15,181 WARNING [conformer_encoder.py:139] Using legacy_rel_pos and it will be deprecated in the future.
2023-12-11 18:13:15,181 WARNING [conformer_encoder.py:139] Using legacy_rel_pos and it will be deprecated in the future.
2023-12-11 18:13:15,181 WARNING [conformer_encoder.py:139] Using legacy_rel_pos and it will be deprecated in the future.
2023-12-11 18:13:15,181 WARNING 

In [150]:
import pandas as pd
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import time
import numpy as np
import soundfile as sf

col_names = ['path', 'text', 'time', 'sr']
preds = ""
target = ""

for i, chunk in enumerate(chunks_array):
    start_time = time.time()
    speech = np.array([])
    duration=split_sec['end'][i]*1000-split_sec['start'][i] * 1000
    for _, row in df.iterrows():
        start_sample = row['start']-split_sec['start'][i] * 1000
        end_sample = row['end']-split_sec['start'][i] * 1000
#         print(start_sample, end_sample, duration)
        if(start_sample<0 and end_sample<0):
            continue
        if(start_sample>duration and end_sample>duration):
            continue
#         print("Y")
        speech = np.concatenate([speech, chunk[int(max(0,start_sample))*16:int(min(duration, end_sample))*16]])

    print(len(speech))
    
    if len(speech) != 0:
        nbests = speech2text(speech)
        text, *_ = nbests[0]
#         output_filename = f'results/output_chunk{i}.wav'
#         sf.write(output_filename, speech, samplerate=16000)
        if(preds==""):
            preds=(text_normalizer(text))
        else:
            preds += " " + (text_normalizer(text))

    print(i, "/500")
    elapsed_time = time.time() - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")


16640
0 /500
Time taken: 5.47 seconds
9920


2023-12-11 18:13:25,465 WARNING [beam_search.py:476] best hypo length: 9 == max output length: 9
2023-12-11 18:13:25,465 WARNING [beam_search.py:476] best hypo length: 9 == max output length: 9
2023-12-11 18:13:25,465 WARNING [beam_search.py:476] best hypo length: 9 == max output length: 9
2023-12-11 18:13:25,465 WARNING [beam_search.py:476] best hypo length: 9 == max output length: 9
2023-12-11 18:13:25,465 WARNING [beam_search.py:476] best hypo length: 9 == max output length: 9
2023-12-11 18:13:25,465 WARNING [beam_search.py:476] best hypo length: 9 == max output length: 9
2023-12-11 18:13:25,465 WARNING [beam_search.py:476] best hypo length: 9 == max output length: 9
2023-12-11 18:13:25,465 WARNING [beam_search.py:476] best hypo length: 9 == max output length: 9
2023-12-11 18:13:25,465 WARNING [beam_search.py:476] best hypo length: 9 == max output length: 9
2023-12-11 18:13:25,465 WARNING [beam_search.py:476] best hypo length: 9 == max output length: 9
2023-12-11 18:13:25,465 WARNIN

1 /500
Time taken: 3.11 seconds
15200
2 /500
Time taken: 4.67 seconds
11040
3 /500
Time taken: 3.13 seconds
23840
4 /500
Time taken: 5.49 seconds
39040
5 /500
Time taken: 6.20 seconds
23360
6 /500
Time taken: 7.02 seconds
9920
7 /500
Time taken: 2.81 seconds
17120
8 /500
Time taken: 5.09 seconds
36000
9 /500
Time taken: 5.24 seconds
72800
10 /500
Time taken: 8.48 seconds
56320
11 /500
Time taken: 9.73 seconds
20800
12 /500
Time taken: 6.20 seconds
14240
13 /500
Time taken: 4.76 seconds
53920
14 /500
Time taken: 7.27 seconds
26400
15 /500
Time taken: 4.19 seconds
27680
16 /500
Time taken: 4.84 seconds
31200
17 /500
Time taken: 5.40 seconds
13600


/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


18 /500
Time taken: 4.23 seconds
43680
19 /500
Time taken: 7.41 seconds
36960
20 /500
Time taken: 7.15 seconds
19040
21 /500
Time taken: 5.89 seconds
37120
22 /500
Time taken: 10.08 seconds
27840
23 /500
Time taken: 10.98 seconds
22880
24 /500
Time taken: 5.62 seconds
73280
25 /500
Time taken: 11.30 seconds
8800


2023-12-11 18:16:01,439 WARNING [beam_search.py:476] best hypo length: 8 == max output length: 8
2023-12-11 18:16:01,439 WARNING [beam_search.py:476] best hypo length: 8 == max output length: 8
2023-12-11 18:16:01,439 WARNING [beam_search.py:476] best hypo length: 8 == max output length: 8
2023-12-11 18:16:01,439 WARNING [beam_search.py:476] best hypo length: 8 == max output length: 8
2023-12-11 18:16:01,439 WARNING [beam_search.py:476] best hypo length: 8 == max output length: 8
2023-12-11 18:16:01,439 WARNING [beam_search.py:476] best hypo length: 8 == max output length: 8
2023-12-11 18:16:01,439 WARNING [beam_search.py:476] best hypo length: 8 == max output length: 8
2023-12-11 18:16:01,439 WARNING [beam_search.py:476] best hypo length: 8 == max output length: 8
2023-12-11 18:16:01,439 WARNING [beam_search.py:476] best hypo length: 8 == max output length: 8
2023-12-11 18:16:01,439 WARNING [beam_search.py:476] best hypo length: 8 == max output length: 8
2023-12-11 18:16:01,439 WARNIN

26 /500
Time taken: 2.76 seconds
16800
27 /500
Time taken: 4.41 seconds
21600
28 /500
Time taken: 5.67 seconds
13120
29 /500
Time taken: 3.82 seconds
71200
30 /500
Time taken: 10.80 seconds
19520
31 /500
Time taken: 7.45 seconds
12320
32 /500
Time taken: 4.32 seconds
17600
33 /500
Time taken: 5.22 seconds
16480
34 /500
Time taken: 4.97 seconds
38400
35 /500
Time taken: 6.48 seconds
26560
36 /500
Time taken: 6.33 seconds
38560
37 /500
Time taken: 6.53 seconds
32000
38 /500
Time taken: 8.95 seconds
31360
39 /500
Time taken: 8.91 seconds
22720
40 /500
Time taken: 8.98 seconds
74400
41 /500
Time taken: 9.56 seconds
43200
42 /500
Time taken: 6.77 seconds
14880
43 /500
Time taken: 5.06 seconds
14880
44 /500
Time taken: 5.72 seconds
28160
45 /500
Time taken: 5.01 seconds
16320
46 /500
Time taken: 5.41 seconds
18880
47 /500
Time taken: 5.96 seconds
22080
48 /500
Time taken: 7.23 seconds
9920
49 /500
Time taken: 2.65 seconds
61280
50 /500
Time taken: 5.94 seconds


she was tired of other things she tried this morning an air or two upon the piano sang a simple song in a sweet but slightly metallic voice and then seating herself by the open window read philips letter well mother said the young student looking up with a shade of impatience i hope thee told the elders that father and i are responsible for the piano and that much as thee loves music thee is never in the room when it is played i heard father tell cousin abner that he was whipped so often for whistling when he was a boy that he was determined to have what compensation he could get now thy ways greatly try me ruth and all thy relations is thy father willing thee should go away to a school of the worlds people i have not asked him ruth replied with a look that might imply that she was one of those determined little bodies who first made up her own mind and then compelled others to make up theirs in accordance with hers mother im going to study medicine margaret bolton almost lost for a moment her habitual placidity

In [151]:
print(preds)

THE CHIEF WEVE SEEN THAT WEVE SO ID SAY THIRD OF OTHER THINGS SHE TRIED THIS MORNING AND ERIC WERE GOING TO  ILL TALK TO YOU ABOUT SO THATS THE FROM THE PNL AND STAYING A SIMPLE SONG AND ITS SWEET BUT SLIGHTLY METALLIC VOIDS AND THEN YOURSELVES BY THE OPEN WINDOW RED PHILIPS PLATTER WELL THE IS THERE  ARE THERE ANY SO THE YOUNG STUDENT LOOKING AT  WITH THE SHADOW IN PATIENTS I HOPE YOU TOLD US ELDERS THAT FATHER AND I ARE RESPONSIBLE FOR THE AND ID LIKE TO KNOW AND THAT MUCH OF THE LLOYDS MUSIC IS NEVER IN THE ROOM WHEN IT IS PLAYED I HEARD FATHER TELKONES AND AVENUE THAT HES WITH WHITBREAD SO ILL START WITH AND FOR WHISTLELING WHEN HE WAS A BOY THAT HE WAS DETERMINED TO HAVE WHAT COMPENSATION HE COULD GET NOW ID LIKE TO TURN THE SO THATS THE ITS GREAT FOR US STATEMENTS MADE BY MANAGEMENT ITS TRIMMING ROOTS AND ALL THATS  IM NOT FATHER WITH SO IT WAS I THINK THEYRE GOING TO BE IT SHOULD GO THE WAY TO A SCHOOL OF THE WORLDS PEOPLE I HAVE NOT ASKED HIM REREPLIED WITH A LOOK THAT MIGHT SI

In [152]:
from torchmetrics.text import CharErrorRate, WordErrorRate
target="she tried this morning an air or two upon the piano sang a simple song in a sweet but slightly metallic voice and then seating herself by the open window read philips letter"
cer = CharErrorRate()
print("CER:", cer(preds, target))
wer = WordErrorRate()
print("WER:", wer(preds, target))

CER: tensor(7.4393)
WER: tensor(7.9091)
